<a href="https://colab.research.google.com/github/2303A52114/GENERATIVE-AI/blob/main/ASSIGNMENT_6_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load dataset
file_path = "Housing.csv"
df = pd.read_csv(file_path)

# Identify categorical and numerical columns
categorical_columns = ['mainroad', 'guestroom', 'basement', 'hotwaterheating',
                       'airconditioning', 'prefarea', 'furnishingstatus']
numerical_columns = [col for col in df.columns if col not in categorical_columns + ['price']]

# One-hot encode categorical variables
df = pd.get_dummies(df, columns=categorical_columns, drop_first=True)

# Separate features and target variable
X = df.drop(columns=['price'])
y = df['price']

# Normalize numerical features
scaler = StandardScaler()
X[numerical_columns] = scaler.fit_transform(X[numerical_columns])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the ANN model
model = Sequential([
    Dense(15, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(20, activation='relu'),
    Dense(25, activation='relu'),
    Dense(20, activation='relu'),
    Dense(15, activation='relu'),
    Dense(1, activation='linear')
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model
model.fit(X_train, y_train, epochs=150, batch_size=32, validation_data=(X_test, y_test), verbose=1)

# Evaluate the model
train_loss, train_mae = model.evaluate(X_train, y_train, verbose=1)
test_loss, test_mae = model.evaluate(X_test, y_test, verbose=1)
print(f'Training MAE: {train_mae}, Testing MAE: {test_mae}')

# Save the model
model.save("/mnt/data/housing_price_model.h5")
print("Model saved as 'housing_price_model.h5'.")

# Load and use the saved model
loaded_model = tf.keras.models.load_model("/mnt/data/housing_price_model.h5")
predictions = loaded_model.predict(X_test[:5])

# Display sample predictions
print("Sample Predictions:")
print(predictions)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/150
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 40ms/step - loss: 26544959389696.0000 - mae: 4810728.0000 - val_loss: 30129992499200.0000 - val_mae: 5007536.5000
Epoch 2/150
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 24251761426432.0000 - mae: 4651609.0000 - val_loss: 30129988304896.0000 - val_mae: 5007536.0000
Epoch 3/150
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 25648145891328.0000 - mae: 4756376.5000 - val_loss: 30129982013440.0000 - val_mae: 5007535.5000
Epoch 4/150
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 24944643670016.0000 - mae: 4695528.5000 - val_loss: 30129967333376.0000 - val_mae: 5007534.0000
Epoch 5/150
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 24568443961344.0000 - mae: 4654930.5000 - val_loss: 30129919098880.0000 - val_mae: 5007529.0000
Epoch 6/150
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 26542526693376.0000 - mae: 4803626.0000 - val_loss: 30129818435584.0000 - val_mae: 5007519.5000
Epoch 7/150
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 2441

Training MAE: 821157.4375, Testing MAE: 1024091.25


Model saved as 'housing_price_model.h5'.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
Sample Predictions:
[[4588195.5]
 [6374317. ]
 [3465519. ]
 [4786891. ]
 [2411119.5]]
